In [ ]:
import datashader
from holoviews.plotting.util import process_cmap
from numpy import log
from pyopenms import *
import pandas as pd
import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews import dim
hv.extension('bokeh')


i = 0

def modify_doc(doc):
    """Add a plotted function to the document.

    Arguments:
        doc: A bokeh document to which elements can be added.
    """
    exp = MSExperiment()
    MzMLFile().load("/Volumes/Data/UPS1/mzML/UPS1_5000amol_R1.mzML", exp)
    cols = ["RT", "mzarray", "intarray"]
    expandcols = ["RT", "mz", "int"]
    #spectraarr = np.fromiter(((spec.getRT(), point[0], point[1]) for spec in exp if spec.getMSLevel() == 1 for point in zip(*spec.get_peaks())), dtype=['f','f','f'])
    #spectradf = pd.DataFrame(data=((spec.getRT(), *spec.get_peaks()) for spec in exp if spec.getMSLevel() == 1), columns=cols)
    spectradf = pd.DataFrame(data=((spec.getRT(), point[0], point[1]) for spec in exp if spec.getMSLevel() == 1 for point in zip(*spec.get_peaks())), columns=expandcols)

    points = hv.Points(spectradf, kdims=['RT', 'mz'], vdims=['int'], label = "MS1").opts(
        fontsize={'title': 16, 'labels': 14, 'xticks': 6, 'yticks': 12},
        color=log(dim('int')),
        colorbar=True,
        cmap='Magma',
        width=1000,
        height=1000,
        tools=['hover'])

    # Unfortunately datashade cannot transfer interactiveness (e.g. hover) or colorbars
    # see https://anaconda.org/jbednar/datashade_vs_rasterize/notebook
    #shade = hd.datashade(points,cmap=process_cmap("blues", provider="bokeh"), alpha=250, min_alpha=10).opts(
    #    plot=dict(
    #        width=1000,
    #        height=1000))#, color_key=colors)
    raster = hd.rasterize(points,cmap=process_cmap("blues", provider="bokeh"),aggregator=datashader.sum('int'),cnorm='log',alpha=50, min_alpha=10).opts(
        tools=['hover']).opts(
        plot = dict(
            width=1000,
            height=1000)
    )
    #selection = hv.streams.Selection1D(source=points)

    # 2. Instead of Jupyter's automatic rich display, render the object as a bokeh document
    #hv.renderer('bokeh').server_doc(hd.dynspread(shade), doc=doc)
    hv.renderer('bokeh')
    hd.dynspread(raster)
